In [ ]:
%tensorflow_version 2.x

## Testing for GPU

To Test if you have GPU set up

Run the Cell below

if no GPU is found press Runtime (in the menu at the top) and choose "Change Runtime Type" to GPU
 

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!nvidia-smi

Mon Jun 29 07:50:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    36W / 250W |    353MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ls

sample_data/


In [ ]:
%matplotlib inline
import os

import tensorflow as tf

import tensorflow.keras

from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D


In [ ]:
batch_size = 128
num_classes = 10
epochs = 10
learning_rate = 0.001

# input image dimensions
img_rows, img_cols = 28, 28

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

In [ ]:
input_shape

(28, 28, 1)

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.
x_test /= 255.

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [ ]:
Inp=Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(Inp)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(64, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)
output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

In [ ]:
model = Model(Inp,output)

## Calculating the number of parameters for Convolution Networks

Formula is (filters x (kernel_height x kernel_width) x inputs) 1 x filters = parameters

Conv_01 = 32 Filters , 3x3 Kernel = (32 x (3x3) x 1 input) + 1bias x filters = 320 parameters  
Conv_02 = 64 Filters , 3x3 Kernel = (64 x (3x3) x 32 inputs) + 1bias x filters = 18496 parameters 

Conv_03 = 64 Filters , 3x3 Kernel = (64 x (3x3) x 64 inputs) + 1bias x filters = 36928 parameters 

Dense_1 = (576 inputs * 64 units)+ 64 bias = 36928 
Dense_2 = (64 inputs * 10 units) +10 bias = 650


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
Conv_01 (Conv2D)             (None, 26, 26, 32)        320       
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (None, 13, 13, 32)        0         
_________________________________________________________________
Conv_02 (Conv2D)             (None, 11, 11, 64)        18496     
_________________________________________________________________
MaxPool_02 (MaxPooling2D)    (None, 5, 5, 64)          0         
_________________________________________________________________
Conv_03 (Conv2D)             (None, 3, 3, 64)          36928     
_________________________________________________________________
Flatten_01 (Flatten)         (None, 576)               0     

In [ ]:
#optimizer
opt = tf.keras.optimizers.Adam(lr = learning_rate)

model.compile(loss= tf.keras.losses.categorical_crossentropy,
              optimizer= opt,
              metrics=['accuracy'])

In [ ]:
hist = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks = None,
          validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 3s 6ms/step - loss: 0.3927 - accuracy: 0.8773 - val_loss: 0.0630 - val_accuracy: 0.9804
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1215 - accuracy: 0.9649 - val_loss: 0.0458 - val_accuracy: 0.9853
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0885 - accuracy: 0.9744 - val_loss: 0.0391 - val_accuracy: 0.9873
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0690 - accuracy: 0.9809 - val_loss: 0.0352 - val_accuracy: 0.9885
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0594 - accuracy: 0.9829 - val_loss: 0.0438 - val_accuracy: 0.9870
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0518 - accuracy: 0.9854 - val_loss: 0.0293 - val_accuracy: 0.9907
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0449 - accuracy: 0.9876 - val_loss: 0.0283 - val_accuracy: 0.9910
Epoch 

In [ ]:
model.save_weights('basic_cnn_weights.h5')

In [ ]:
model.save('./mnist_cnn_model.h5')

In [ ]:
ls

basic_cnn_weights.h5  mnist_cnn_model.h5  sample_data/  saved_models/


### Saved_model format

In [ ]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))


model.save(saved_model_path, save_format='tf') 
# saved_model_path

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saved_models/1593417662/assets


In [ ]:
ls saved_models/1593417662/


assets/  saved_model.pb  variables/


In [ ]:
ls

## Let's test it out

In [ ]:
Inp=Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(Inp)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(64, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)
output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

In [ ]:
model = Model(Inp,output)

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer= opt,
              metrics=['accuracy'])



In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.3054 - accuracy: 0.0976


[2.30535888671875, 0.09759999811649323]

In [ ]:
model.load_weights('basic_cnn_weights.h5')

In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0282 - accuracy: 0.9926


[0.028185291215777397, 0.9926000237464905]

## Saving your model to GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
ls

In [ ]:
!ls 'gdrive/My Drive/colab_stuff/'

In [ ]:
!cp './basic_cnn_weights.h5' 'gdrive/My Drive/colab_stuff/weight.h5'

In [ ]:
!mkdir test_dir

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
%cd /content

In [ ]:
Inp=Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(Inp)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(128, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Conv2D(256, (3, 3), activation='relu',name = 'Conv_04')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)

x = Dropout(0.25,name = 'Dropout_01')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(128, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)
output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

In [ ]:
model = Model(Inp,output)

## Calculating the number of parameters for Convolution Networks

Formula is (filters x (kernel_height x kernel_width) x inputs) 1 x filters = parameters

Conv_01 = 32 Filters , 3x3 Kernel = (32 x (3x3) x 1 input) + 1bias x filters = 320 parameters  
Conv_02 = 64 Filters , 3x3 Kernel = (64 x (3x3) x 32 inputs) + 1bias x filters = 18496 parameters   
Conv_03 = 128 Filters , 3x3 Kernel = (128 x (3x3) x 64 input) + 1bias x filters = 73856 parameters    
Conv_04 = 256 Filters , 3x3 Kernel = (256 x (3x3) x 128 inputs) + 1bias x filters = 295168 parameters 

Dense_1 = (4096 inputs * 128 units)+ 128 bias = 524,416  
Dense_2 = (128 inputs * 10 units) +10 bias = 1290

In [ ]:
model.summary()